In [ ]:
# pip install nilearn pandas matplotlib pybids
# uncomment if you haven't downloaded these already,
# otherwise you can leave it commented out because it takes too long to run tbh...

In [ ]:
import numpy as np
import pandas as pd
from bids import BIDSLayout
from nilearn.glm.first_level import FirstLevelModel
from nilearn import plotting
import matplotlib.pyplot as plt
import os

# PATH CONFIG
data_dir = r'C:\Users\Ria\OneDrive\COGS 108\ds004920' # ***REPLACE THIS PATH WITH UR OWN***
# data_dir folder should still be titled 'ds004920' if u downloaded it from github, but filepath should be changed to ur own
sub_id = '1001' # starting with subject 1001. replace this with other subject IDs as needed
run_id = 1 # starting with run 1. replace with other run IDs as needed
print(f"Targeting dataset at: {data_dir}") # just returns the path to the dataset

# initialize BIDSLayout to ensure that it indexes the derivatives folder
try:
    layout = BIDSLayout(data_dir, derivatives=True) 
    subjects = layout.get_subjects()
    if sub_id not in subjects:
        print(f"Subject {sub_id} not found in dataset.") # check if subject exists in dataset
    print(f"BIDS Layout initialized. Found {len(subjects)} subjects.")
except Exception as e:
    print(f"ERROR in initializing BIDS Layout. Check if the path is correct and the dataset is valid. BIDS: {e}")
    # if u see this error, it might be bc the path is wrong or the dataset isn't BIDS formatted
    # stop execution if BIDSLayout fails to avoid subsequent errors
    raise

In [ ]:
# find the raw events file for the targeted subject
events_files = layout.get(subject=sub_id, task='sharedreward', run=run_id, suffix='events', extension='tsv')

if not events_files:
    raise FileNotFoundError("ERROR: Could not find raw events.tsv file. Check your task name.")
    # if u see this error, double check that ur task name correctly named 'sharedreward' in the events file
    
# load events file into dataframe
events_df = pd.read_csv(events_files[0].path, sep='\t')
print(f"Events loaded from: {events_files[0].path}")
print("Trial Types found in events_df:", events_df['trial_type'].unique().tolist())
# this just checks what trial types are in the events file

In [ ]:
# 1. LOCATE PREPROCESSED IMAGE
# use PyBIDS search for the now-preprocessed BOLD file
func_files = layout.get(subject=sub_id, 
                        task='sharedreward', 
                        run=run_id,
                        desc='preproc', 
                        space='MNI152NLin2009cAsym', 
                        suffix='bold', 
                        extension='nii.gz',  # file should be named nii.gz and should be in the same folder
                        return_type='file')

if not func_files:
    raise FileNotFoundError("ERROR: Couldn't find preprocessed BOLD file in derivatives.")
# if u see this error, double check that fMRIPrep was run successfully on the dataset
    
func_img = func_files[0]
print(f"Preprocessed image found: {func_img}")

# 2. LOCATE FMRIPREP CONFOUNDS FILE
confounds_files = layout.get(subject=sub_id, 
                             task='sharedreward', 
                             run=run_id, 
                             desc='confounds', 
                             suffix='timeseries', 
                             extension='tsv',
                             return_type='file')

if not confounds_files:
    raise FileNotFoundError("ERROR: Couldn't find fMRIPrep confounds file.")
# if u see this error, double check that fMRIPrep was run successfully on the dataset

confounds_path = confounds_files[0]
confounds_df = pd.read_csv(confounds_path, sep='\t')
print(f"Confounds file found: {confounds_path}")

# 3. SELECT NUISANCE REGRESSORS
motion_confounds = ['trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z'] # 6 motion parameters (translation and rotation)
# filter DataFrame for only the required columns
selected_confounds_df = confounds_df[motion_confounds]

# 4. GET TR (repetition time)
t_r = layout.get_tr(func_img) 
print(f"Repetition Time (TR): {t_r} seconds")


# 5. INITIALIZE AND FIT GLM
# include standardize=False for fMRIPrep data (bc it's already normalized/scaled)
fmri_glm = FirstLevelModel(t_r=t_r,
                           noise_model='ar1',
                           standardize=False, 
                           hrf_model='glover',
                           drift_model='cosine',
                           high_pass=0.01)

print("Fitting GLM with fMRIPrep confounds...")
fmri_glm = fmri_glm.fit(func_img, events=events_df, confounds=selected_confounds_df) 
print("GLM FIT SUCCESSFUL!")

In [ ]:
# get the design matrix from the fitted model
design_matrix = fmri_glm.design_matrices_[0]

# print column names to confirm contrast strings are correct
print("\n--- Model Conditions (Regressors) ---")
print(design_matrix.columns.tolist())

# define contrasts using exact column names
contrasts = {
    # 1. Friend vs. Computer
    'Friend_vs_Computer': '(event_friend_reward + event_friend_punish + event_friend_neutral) - (event_computer_reward + event_computer_punish + event_computer_neutral)',

    # 2. Friend vs. Stranger
    'Friend_vs_Stranger': '(event_friend_reward + event_friend_punish + event_friend_neutral) - (event_stranger_reward + event_stranger_punish + event_stranger_neutral)',

    # 3. Social vs. Non-Social
    'Social_vs_Computer': '0.5*(event_friend_reward + event_friend_punish + event_friend_neutral + event_stranger_reward + event_stranger_punish + event_stranger_neutral) - (event_computer_reward + event_computer_punish + event_computer_neutral)'
}

print("\n✅ Contrasts defined.")
print("Plotting Design Matrix (Now includes Motion Confounds):")

# plot design matrix
plotting.plot_design_matrix(design_matrix)
plt.title("Design Matrix (fMRIPrep Confounds Included)")
plt.show()

In [ ]:
# FRIEND > COMPUTER
z_map_fc = fmri_glm.compute_contrast(contrasts['Friend_vs_Computer'], output_type='z_score')
print("\nPlotting Friend > Computer Contrast:")

# plot Z-score map
plotting.plot_stat_map(z_map_fc, threshold=3.0, display_mode='z', cut_coords=5,
                       title='Friend > Computer (Z > 3.0) - MNI Space', black_bg=True)
plt.show()

In [ ]:
# FRIEND > STRANGER
z_map_fs = fmri_glm.compute_contrast(contrasts['Friend_vs_Stranger'], output_type='z_score')
print("Plotting Friend > Stranger Contrast:")

# plot Z-score map
plotting.plot_stat_map(z_map_fs, threshold=3.0, display_mode='z', cut_coords=5, 
                       title='Friend > Stranger (Z > 3.0) - MNI Space', black_bg=True)  
plt.show()

In [ ]:
# CONTINUE COPY PASTING THE ABOVE CODE BLOCK FOR OTHER COMPARISONS AS NEEDED
# just replace the variables and contrast names accordingly